In [2]:
import numpy as np
import pandas as pd

In [178]:
base = pd.read_excel('C:/Users/conde/Study/multivariada/dados/complementar - livro metodos multivariados/clientes.xlsx')
comunidade = pd.read_excel('C:/Users/conde/Study/multivariada/dados/complementar - livro metodos multivariados/comunidade.xlsx')
base

,Clientes,idade,n_carros,classe_social,potencia,cambio,modelo
0,1,20,1,A,baixa,automatico,esporte
1,2,37,3,A,alta,automatico,luxo
2,3,22,2,B,media,automatico,esporte
3,4,26,2,B,alta,automatico,esporte
4,5,45,2,C,media,manual,standard
5,6,42,1,D,baixa,manual,standard


## TRATANDO DUMMIES

In [457]:
# DUMMIES COM ORDEM

def dummie_coluna(coluna, prefix='dummie', order=[], nominal=True):
    n = coluna.nunique()

    if nominal: #nominal
        codigo_dummies = np.identity(n-1, dtype=int) #matriz identidade
        classes = coluna.unique() if not order else order
        #classes = coluna.unique().reshape(-1, 1) if not order else order
        
    else: #ordinal
        codigo_dummies = np.triu(np.ones((n-1, n-1), dtype=int)) #matriz triangular superior
        classes = order

    linha_zeros = np.zeros((1, n-1), dtype=int)
    matriz_dummies = np.vstack((codigo_dummies, linha_zeros))

    nome_colunas = [f'{prefix}_{str(i)}' for i in range(1, n)]
    dummies = pd.DataFrame(data=matriz_dummies, columns=nome_colunas)
    dummies.insert(0, 'classes', classes)  # Adiciona a coluna de classes
    
    base_dummies = pd.DataFrame(coluna).merge(dummies, how='left', left_on=coluna.name, right_on='classes')
    base_dummies.drop(columns=['classes'], inplace=True)
    
    return base_dummies

In [287]:
def dummies(base, nominal = {}, excluir_coluna_origem=False):
    """
    Cria dummies apenas de colunas categoricas
    
    base: (DataFrame) recebe um data frame da base, com ou sem, colunas numeriacas. E seleciona apenas as colunas categoricas, se quiser dummies de colunas numericas faça use dummie_coluna()
    nominal: (Dic) {'coluna': (nominal (True ou False), [lista da ordem])}
    excluir_coluna_origem: (bool) True ex
    """


    especial = base[nominal.keys()]
    padrao = base.select_dtypes(include=['object', 'category']).drop(especial.columns, axis=1)

    base_especial   = [dummie_coluna(especial[coluna], prefix=coluna, order=ordem[1], nominal=ordem[0]) for coluna, ordem in nominal.items()]
    base_padrao     = [dummie_coluna(padrao[coluna], prefix=coluna) for coluna in padrao.columns]

    if nominal == {}:
        base_dummies = pd.concat(base_padrao, axis=1)
    else:
        base_dummies = pd.concat([pd.concat(base_especial, axis=1), pd.concat(base_padrao, axis=1)], axis=1)

    if excluir_coluna_origem:
        base_dummies.drop(list(padrao.columns) + list(especial.columns), axis=1, inplace=True)


    return base_dummies

# DISTANCIAS

## Categoricas

In [296]:
aux = {'potencia': (True, ['media', 'alta', 'baixa'])}
BaseDummies = dummies(base, aux, True)
BaseDummies

,potencia_1,potencia_2,classe_social_1,classe_social_2,classe_social_3,cambio_1,modelo_1,modelo_2
0,0,0,1,0,0,1,1,0
1,0,1,1,0,0,1,0,1
2,1,0,0,1,0,1,1,0
3,0,1,0,1,0,1,1,0
4,1,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0


In [304]:
def qParecenca_matriz(dummies, indv_x, indv_y):
    #"""
    #linhas sao referentes a X e colunas a Y
    #"""
    marginal = [dummies.shape[1] - dummies.iloc[indv_x].sum(), dummies.iloc[indv_x].sum()]
    soma = dummies.iloc[[indv_x, indv_y]].sum()
    iguais = [(soma == 0).sum(), (soma == 2).sum()]

    matriz = np.array([  
                [iguais[1], marginal[1] - iguais[1]], 
                [marginal[0] - iguais[0], iguais[0]]
            ])

    return matriz

In [418]:
def qmConcordancia(matriz):
    ad = (matriz[0][0] + matriz[1][1])

    return ad/matriz.sum()

def qmDiscordancia(matriz):
    bc = (matriz[0][1] + matriz[1][0])
    
    return bc/matriz.sum()

def qmJaccard(matriz):
    #pelo menos 1 par
    a = matriz[0][0]
    abc = matriz[0][0] + matriz[0][1] + matriz[1][0]

    return a/abc

def qmBC(matriz):
    #medida de discimilariedade
    bc = matriz[0][1] +  matriz[1][0]
    aabc = 2*matriz[0][0] + matriz[0][1] + matriz[1][0]
    
    return bc/aabc

def qmCS(matriz):
    #medida de similariedade
    #1 - BC
    aa = 2*matriz[0][0]
    aabc = 2*matriz[0][0] + matriz[0][1] + matriz[1][0]

    return aa/aabc

In [344]:
def qDistancia_Matriz(dummies, funcao_distancia=qmConcordancia, index=None):
    nrow = base.shape[0]
    matriz_distancia = np.zeros((nrow, nrow))

    for i in range(nrow):
        for j in range(nrow):
            matriz = qParecenca_matriz(dummies, i, j)
            matriz_distancia[i][j] = funcao_distancia(matriz)


    if index == None:
        index = base.index

    return pd.DataFrame(matriz_distancia, columns=index, index=index)

In [350]:
qDistancia_Matriz(BaseDummies, qmJaccard)

C:\Users\conde\AppData\Local\Temp\ipykernel_7276\2553050838.py:16: RuntimeWarning: invalid value encountered in scalar divide
  return a/abc


,0,1,2,3,4,5
0,1.0,0.400000,0.400000,0.400000,0.0,0.0
1,0.4,1.000000,0.142857,0.333333,0.0,0.0
2,0.4,0.142857,1.000000,0.600000,0.2,0.0
3,0.4,0.333333,0.600000,1.000000,0.0,0.0
4,0.0,0.000000,0.200000,0.000000,1.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.0,NaN


# Numerica

In [382]:
#medidas de distancias
def nmMinkowsky(base, i, j, n=2, m=2):
    """
    m = raiz, n = potencia da diferença
    distancia euclidiana: m=2, n=2
    distancia de manhatam: n=1, m=1
    """
    diferenca = abs(base.iloc[i] - base.iloc[j])**m
    distancia = (sum(diferenca))**(1/n)

    return distancia

def nmBC(base, i, j):
    total = sum(base.iloc[[i, j]].sum())
    distancias = nmMinkowsky(base, i, j, n=1, m=1)

    return distancias/total

In [358]:
#matriz de distancias
def nDistancia_Matriz(base, funcao_distancia, index=None):
    nrow = base.shape[0]
    matriz_distancia = np.zeros((nrow, nrow))

    for i in range(nrow):
        for j in range(nrow):    
            matriz_distancia[i][j] = funcao_distancia(base, i, j)

    if index == None:
        index = base.index

    return pd.DataFrame(matriz_distancia, columns=index, index=index)

In [361]:
nDistancia_Matriz(comunidade.iloc[:,1:], nmBC, index=[1, 2,3,4])

,1,2,3,4
1,0.000000,0.041322,0.875000,0.846154
2,0.041322,0.000000,0.876923,0.848485
3,0.875000,0.876923,0.000000,0.555556
4,0.846154,0.848485,0.555556,0.000000


# Distancia numericas e categoricas

quando temos tanto variaveis categoricas quanto numerica, analisamos elas separadamente e depois juntamos poderando cada categoria de distancia (categorica e numerica) pelo seu numero de variavaies. Existem outros metodos, mas o basico é esse.

Para o metodo basico usamos a distancia euclidiana, pois a medida de dissimilaridade da variavel categorica é a media das distancias euclidianas ao quadrado, entao ajustando a distancia euclidiana das variaveis numericas conseguimos compararar as 2 categorias de distancias

In [378]:
categorico = base.select_dtypes(include=['object', 'category'])
dummies(categorico, excluir_coluna_origem=True)

,classe_social_1,classe_social_2,classe_social_3,potencia_1,potencia_2,cambio_1,modelo_1,modelo_2
0,1,0,0,1,0,1,1,0
1,1,0,0,0,1,1,0,1
2,0,1,0,0,0,1,1,0
3,0,1,0,0,1,1,1,0
4,0,0,1,0,0,0,0,0
5,0,0,0,1,0,0,0,0


In [380]:
qDistancia_Matriz(dummies(categorico, excluir_coluna_origem=True), qmConcordancia)**2

,0,1,2,3,4,5
0,1.000000,0.250000,0.390625,0.250000,0.140625,0.390625
1,0.250000,1.000000,0.140625,0.250000,0.140625,0.140625
2,0.390625,0.140625,1.000000,0.765625,0.250000,0.250000
3,0.250000,0.250000,0.765625,1.000000,0.140625,0.140625
4,0.140625,0.140625,0.250000,0.140625,1.000000,0.562500
5,0.390625,0.140625,0.250000,0.140625,0.562500,1.000000


In [366]:
numerico = base.iloc[:,1:].select_dtypes(include=['int64', 'float64'])
numerico

,idade,n_carros
0,20,1
1,37,3
2,22,2
3,26,2
4,45,2
5,42,1


In [479]:
def trans_MinMax(x):
    return (x - x.min()) / (x.max() - x.min())

numerico_tratamento = numerico.apply(trans_MinMax)
numerico_tratamento

,idade,n_carros
0,0.00,0.0
1,0.68,1.0
2,0.08,0.5
3,0.24,0.5
4,1.00,0.5
5,0.88,0.0


In [481]:
#a funçao de nmMinkowsjy() tem por padrao a distancia euclidiana, entoa estou usnado a distancia euclidiana
numerico_distancia = nDistancia_Matriz(numerico_tratamento, nmMinkowsky)
numerico_distancia

,0,1,2,3,4,5
0,0.000000,1.209297,0.506360,0.554617,1.118034,0.880000
1,1.209297,0.000000,0.781025,0.666033,0.593633,1.019804
2,0.506360,0.781025,0.000000,0.160000,0.920000,0.943398
3,0.554617,0.666033,0.160000,0.000000,0.760000,0.812158
4,1.118034,0.593633,0.920000,0.760000,0.000000,0.514198
5,0.880000,1.019804,0.943398,0.812158,0.514198,0.000000


In [472]:
categorico = base.select_dtypes(include=['object', 'category'])
categorico

,classe_social,potencia,cambio,modelo
0,A,baixa,automatico,esporte
1,A,alta,automatico,luxo
2,B,media,automatico,esporte
3,B,alta,automatico,esporte
4,C,media,manual,standard
5,D,baixa,manual,standard


In [474]:
lista = {   
            'classe_social': (False, ['A', 'B', 'C', 'D']),
            'potencia': (False, ['media', 'alta', 'baixa'])
        }

categorico_tratamento = dummies(categorico, lista, excluir_coluna_origem=True)
categorico_tratamento

,classe_social_1,classe_social_2,classe_social_3,potencia_1,potencia_2,cambio_1,modelo_1,modelo_2
0,1,1,1,0,0,1,1,0
1,1,1,1,0,1,1,0,1
2,0,1,1,1,1,1,1,0
3,0,1,1,0,1,1,1,0
4,0,0,1,1,1,0,0,0
5,0,0,0,0,0,0,0,0


In [476]:
categorico_distancia = qDistancia_Matriz(categorico_tratamento, qmDiscordancia)
categorico_distancia

,0,1,2,3,4,5
0,0.000,0.375,0.375,0.250,0.750,0.625
1,0.375,0.000,0.500,0.375,0.625,0.750
2,0.375,0.500,0.000,0.125,0.375,0.750
3,0.250,0.375,0.125,0.000,0.500,0.625
4,0.750,0.625,0.375,0.500,0.000,0.375
5,0.625,0.750,0.750,0.625,0.375,0.000


In [483]:
2*(numerico_distancia**2)/2 + 4*(categorico_distancia)

,0,1,2,3,4,5
0,0.0000,2.9624,1.7564,1.3076,4.2500,3.2744
1,2.9624,0.0000,2.6100,1.9436,2.8524,4.0400
2,1.7564,2.6100,0.0000,0.5256,2.3464,3.8900
3,1.3076,1.9436,0.5256,0.0000,2.5776,3.1596
4,4.2500,2.8524,2.3464,2.5776,0.0000,1.7644
5,3.2744,4.0400,3.8900,3.1596,1.7644,0.0000
